# 模型参数的访问、初始化和共享

在[“线性回归的简洁实现”]一节中，我们通过`init`模块来初始化模型的全部参数。我们也介绍了访问模型参数的简单方法。本节将深入讲解如何访问和初始化模型参数，以及如何在多个层之间共享同一份模型参数。

我们先定义一个与上一节中相同的含单隐藏层的多层感知机。我们依然使用默认方式初始化它的参数，并做一次前向计算。

In [ ]:
import tensorflow as tf
import numpy as np
print(tf.__version__)

In [ ]:
net = tf.keras.models.Sequential()
net.add(tf.keras.layers.Flatten())
net.add(tf.keras.layers.Dense(256,activation=tf.nn.relu))
net.add(tf.keras.layers.Dense(10))

X = tf.random.uniform((2,20))
Y = net(X)
Y

## 4.2.1 access model parameters

对于使用`Sequential`类构造的神经网络，我们可以通过weights属性来访问网络任一层的权重。回忆一下上一节中提到的`Sequential`类与`tf.keras.Model`类的继承关系。对于`Sequential`实例中含模型参数的层，我们可以通过`tf.keras.Model`类的`weights`属性来访问该层包含的所有参数。下面，访问多层感知机`net`中隐藏层的所有参数。索引0表示隐藏层为`Sequential`实例最先添加的层。

In [ ]:
net.weights[0], type(net.weights[0])

## 4.2.2 initialize params

我们在[“数值稳定性和模型初始化”]一节中描述了模型的默认初始化方法：权重参数元素为[-0.07, 0.07]之间均匀分布的随机数，偏差参数则全为0。但我们经常需要使用其他方法来初始化权重。在下面的例子中，我们将权重参数初始化成均值为0、标准差为0.01的正态分布随机数，并依然将偏差参数清零。

















这是一个自定义的线性神经网络类，具有特定的初始化设置。让我详细解释其结构和特点：

1. **类的定义**：
```python
class Linear(tf.keras.Model):
```

- 继承自tf.keras.Model
- 实现了一个两层的线性网络

2. **第一个全连接层**：
```python
self.d1 = tf.keras.layers.Dense(
    units=10,
    activation=None,
    kernel_initializer=tf.random_normal_initializer(mean=0,stddev=0.01),
    bias_initializer=tf.zeros_initializer()
)
```

特点：
- 10个神经元
- 无激活函数（线性）
- 权重初始化：使用均值为0，标准差为0.01的正态分布
- 偏置初始化：全部初始化为0

3. **第二个全连接层**：
```python
self.d2 = tf.keras.layers.Dense(
    units=1,
    activation=None,
    kernel_initializer=tf.ones_initializer(),
    bias_initializer=tf.ones_initializer()
)
```

特点：
- 1个神经元（输出层）
- 无激活函数（线性）
- 权重初始化：全部初始化为1
- 偏置初始化：全部初始化为1

4. **前向传播**：
```python
def call(self, input):
    output = self.d1(input)
    output = self.d2(output)
    return output
```

- 数据依次通过两个全连接层
- 保持线性特性（没有非线性激活函数）

这个网络的特点：
1. **完全线性**：
   - 没有使用任何非线性激活函数
   - 整个网络实现的是线性变换

2. **特殊的初始化策略**：
   - 第一层使用随机正态分布初始化
   - 第二层使用全1初始化
   - 体现了不同的参数初始化方法

3. **简单的结构**：
   - 输入 -> 10个神经元 -> 1个输出
   - 适合简单的回归任务


In [4]:
class Linear(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.d1 = tf.keras.layers.Dense(
            units=10,
            activation=None,
            kernel_initializer=tf.random_normal_initializer(mean=0,stddev=0.01),
            bias_initializer=tf.zeros_initializer()
        )
        self.d2 = tf.keras.layers.Dense(
            units=1,
            activation=None,
            kernel_initializer=tf.ones_initializer(),
            bias_initializer=tf.ones_initializer()
        )

    def call(self, input):
        output = self.d1(input)
        output = self.d2(output)
        return output


In [5]:
net = Linear()
net(X)
net.get_weights()

[array([[ 7.77104753e-04,  2.04039668e-03, -2.09118426e-02,
          8.41928180e-03,  2.46993708e-03, -7.85500556e-03,
         -1.02335252e-02, -2.48877835e-02, -5.46165043e-03,
         -2.80203912e-02],
        [ 6.11077482e-03, -3.60709266e-03, -1.50499190e-03,
         -8.58556665e-03, -8.74759443e-03,  5.21494122e-03,
         -2.61842622e-03, -1.56951975e-02, -2.11102404e-02,
         -2.13669497e-03],
        [ 1.47113735e-02, -1.87969767e-02,  5.44861308e-04,
          8.79167393e-03, -7.01406272e-03, -2.23390758e-03,
         -7.23033911e-03,  1.33407116e-02, -1.31247956e-02,
          3.71440337e-03],
        [-1.86755485e-03,  2.76029343e-03,  4.24003601e-03,
         -7.66812405e-03, -1.18439202e-03,  2.97244638e-03,
         -1.18275480e-02, -6.79787016e-03, -4.63147834e-03,
         -6.81351637e-03],
        [ 5.22201788e-03, -4.70648939e-03,  7.02407025e-03,
         -7.97325931e-03,  5.82225854e-03, -5.85760223e-03,
         -1.20407308e-03, -1.72030907e-02,  4.563129

## 4.2.3 define initializer

可以使用`tf.keras.initializers`类中的方法实现自定义初始化。















这段代码展示了如何使用自定义初始化器来创建神经网络层。让我详细解释：

1. **自定义初始化函数**：
```python
def my_init():
    return tf.keras.initializers.Ones()
```

- 定义了一个返回初始化器的函数
- 这里返回了一个全1初始化器
- 这是一个简单的工厂函数模式

2. **创建模型**：
```python
model = tf.keras.models.Sequential()
```

- 创建一个顺序模型
- 用于构建线性堆叠的神经网络

3. **添加层**：
```python
model.add(tf.keras.layers.Dense(64, kernel_initializer=my_init()))
```

- 添加一个具有64个神经元的全连接层
- 使用自定义的初始化器函数
- 所有权重都会被初始化为1

这段代码的特点：
- 展示了如何自定义权重初始化方法
- 通过函数封装使初始化器更灵活
- 可以轻松替换不同的初始化策略


In [6]:
def my_init():
    return tf.keras.initializers.Ones()

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(64, kernel_initializer=my_init()))

In [7]:
Y = model(X)
model.weights[0]

<KerasVariable shape=(20, 64), dtype=float32, path=sequential_1/dense_4/kernel>